In [ ]:
import os
from braindecode.datasets.mne import (create_from_mne_epochs)
import mne
from braindecode.datautil.preprocess import exponential_moving_standardize
from braindecode.datautil.preprocess import MNEPreproc, NumpyPreproc, preprocess
import numpy as np
from braindecode.datautil.windowers import create_windows_from_events
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd


config = {"folder_path": ("/storage/store/data/camcan/camcan47/cc700/"
                     "meg/pipeline/release004/data/aamod_meg_get_fif_00001/"),
          "list_indiv": ['CC110033', 'CC110037', 'CC110045'],
          "maxfilter_calibration_file_path": "sss_cal.dat",
          "empty_room_path": ("/storage/store/data/camcan/camcan47/cc700/meg/"
                              "pipeline/release004/emptyroom/"),
          "event_id": {"Auditory 300Hz": 6, 
                       "Auditory 600Hz": 7,
                       "Auditory 1200Hz": 8, 
                       "Visual Checkerboard": 9},
          "tmin": -0.2, 
          "tmax": 0.5,
          "factor_new": 1e-3,
          "init_block_size": 1000,
         
          "trial_start_offset_seconds": -0.5}

class DataImporter:
    
    def __init__(self, config):
        self.final_epochs_list = []
        self.dataset = []
        
    def _import_epochs_list(self):
        for indiv in config["list_indiv"]:
            raw_path = data_path + indiv + "/passive/passive_raw.fif"
            empty_raw_path = data_path + indiv + "/emptyroom_" + indiv + ".fif"
            config["raw_path"] = raw_path
            config["empty_raw_path"] = empty_raw_path
            new_ep_gen = EpochGenerator(self, config)
            new_ep_gen.compute_epochs()
            self.final_epochs_list.append(new_ep_gen.return_epochs())
            
    def _create_braindecode_dataset(self):
        subject_id = 22
        event_codes = [5, 6, 9, 10, 13, 14]
        self.windows_datasets = create_from_mne_epochs(list_of_epochs,
                                                  window_size_samples=50,
                                                  window_stride_samples=50,
                                                  drop_last_window=False)
        
    def compute(self):
        self._import_epoch_list()
        self._create_braindecode_dataset()
    
    def get(self):
        return(self.windows_datasets)
        
        
class BrainDecodeModel:
    
    def __init__(self, windows_datasets):
        self.preprocessors = None
        self.windows_datasets = None
        
    def _preprocess_data():
        self.preprocessors = [
            # keep only EEG sensors
            MNEPreproc(fn='pick_types', eeg=True, meg=False, stim=False),
            # convert from volt to microvolt, directly modifying the numpy array
            NumpyPreproc(fn=lambda x: x * 1e6)
            # exponential moving standardization
            NumpyPreproc(fn=exponential_moving_standardize, factor_new=factor_new,
                init_block_size=init_block_size)
        ]
        preprocess(dataset, preprocessors)

    def _train_test_split():
        self.splitted = self.windows_dataset.split('session')
        self.train_set = self.splitted['session_T']
        self.valid_set = self.splitted['session_E']


    def _check_if_gpu_available():
        self.cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
        self.device = 'cuda' if cuda else 'cpu'
        if cuda:
            torch.backends.cudnn.benchmark = True
            
    def _define_model():
    
        n_classes=4
        # Extract number of chans and time steps from dataset
        n_chans = train_set[0][0].shape[0]
        input_window_samples = train_set[0][0].shape[1]
        model = ShallowFBCSPNet(
            n_chans,
            n_classes,
            input_window_samples=input_window_samples,
            final_conv_length='auto')

        # Send model to GPU
        if cuda:
            model.cuda()
            
        lr = 0.0625 * 0.01
        weight_decay = 0

        # For deep4 they should be:
        # lr = 1 * 0.01
        # weight_decay = 0.5 * 0.001
        batch_size = 64
        n_epochs = 4

        clf = EEGClassifier(
            model,
            criterion=torch.nn.NLLLoss,
            optimizer=torch.optim.AdamW,
            train_split=predefined_split(valid_set),  # using valid_set for validation
            optimizer__lr=lr,
            optimizer__weight_decay=weight_decay,
            batch_size=batch_size,
            callbacks=[
                "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
            ],
            device=device,
        )
    
    def _train_model():
    
        # These values we found good for shallow network:
        
        # Model training for a specified number of epochs. `y` is None as it is already supplied
        # in the dataset.
        clf.fit(train_set, y=None, epochs=n_epochs)

    def _get_results():
        






# Extract loss and accuracy values for plotting from history object
results_columns = ['train_loss', 'valid_loss', 'train_accuracy', 'valid_accuracy']
df = pd.DataFrame(clf.history[:, results_columns], columns=results_columns,
                  index=clf.history[:, 'epoch'])

# get percent of misclass for better visual comparison to loss
df = df.assign(train_misclass=100 - 100 * df.train_accuracy,
               valid_misclass=100 - 100 * df.valid_accuracy)

plt.style.use('seaborn')
fig, ax1 = plt.subplots(figsize=(8, 3))
df.loc[:, ['train_loss', 'valid_loss']].plot(
    ax=ax1, style=['-', ':'], marker='o', color='tab:blue', legend=False, fontsize=14)

ax1.tick_params(axis='y', labelcolor='tab:blue', labelsize=14)
ax1.set_ylabel("Loss", color='tab:blue', fontsize=14)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

df.loc[:, ['train_misclass', 'valid_misclass']].plot(
    ax=ax2, style=['-', ':'], marker='o', color='tab:red', legend=False)
ax2.tick_params(axis='y', labelcolor='tab:red', labelsize=14)
ax2.set_ylabel("Misclassification Rate [%]", color='tab:red', fontsize=14)
ax2.set_ylim(ax2.get_ylim()[0], 85)  # make some room for legend
ax1.set_xlabel("Epoch", fontsize=14)

# where some data has already been plotted to ax
handles = []
handles.append(Line2D([0], [0], color='black', linewidth=1, linestyle='-', label='Train'))
handles.append(Line2D([0], [0], color='black', linewidth=1, linestyle=':', label='Valid'))
plt.legend(handles, [h.get_label() for h in handles], fontsize=14)
plt.tight_layout()

In [ ]:
from braindecode.datautil import Policy

In [6]:
class EpochGenerator:
    
    def __init__(self, config):
        self.empty_raw_path = config["empty_raw_path"]
        self.raw_path = config["raw_path"]
        self.raw = None
        self.empty_raw = None
        self.maxfilter_raw = None
        self.maxfilter_empty_raw = None
        self.events = None
        self.final_epochs = None
        self.ica = None
        self.reject = None
        
    def _import_data(self):
        self.raw = mne.io.read_raw_fif(self.raw_path)
        self.empty_raw = mne.io.read_raw_fif(self.empty_raw_path)
        
    def _filter_data(self)
        self.raw.fix_mag_coil_types()
        self.maxfilter_raw = mne.preprocessing.maxwell_filter(self.raw, calibration=self.cal_file_path)
        self.maxfilter_raw.filter(l_freq=0.5, h_freq=40)
        self.empty_raw.fix_mag_coil_types()
        self.maxfilter_empty_raw = mne.preprocessing.maxwell_filter(raw, calibration="sss_cal.dat", coord_frame="meg")
        self.maxfilter_empty_raw.filter(l_freq=0.5, h_freq=40)

    def _build_epochs_evoked(self):
        self.events = mne.find_events(self.raw, stim_channel='STI101')
        self.raw.info['projs'] = list()  # remove proj, don't proj while interpolating
        self.epochs = mne.Epochs(self.raw, self.events, config["event_id"], config["tmin"], config["tmax"],
                            baseline=(None, 0), reject=None,
                            verbose=False, detrend=0, preload=True)
        self.reject = get_rejection_threshold(epochs)

    def _create_epochs(raw):
        self.ica = mne.preprocessing.ICA(n_components=30)
        self.ica.fit(raw)
        # raw.load_data()
        self.ica.exclude = []
        eog_indices, eog_scores = self.ica.find_bads_eog(self.raw)
        ecg_indices, ecg_scores = self.ica.find_bads_ecg(self.raw, method='correlation')
        self.ica.exclude = ecg_indices + eog_indices
        self.epochs = mne.Epochs(self.raw, events, event_id, tmin, tmax,
                                 baseline=(None, 0), reject=reject,
                                 verbose=False, detrend=0, preload=True)
    
    def compute(self):
        self._import_data()
        self._filter_data()
        self._build_epochs_evoked()
        self._create_epochs()
        
    def get(self):
        return(self.epochs)

SyntaxError: invalid syntax (<ipython-input-6-da5c6233beb1>, line 4)

In [7]:
class EpochObject:
    def 

SyntaxError: invalid syntax (<ipython-input-7-a577a0dcc470>, line 2)

In [ ]:
def import_data():
    data_path = ()
    
    

In [ ]:
import os
import mne
from autoreject import get_rejection_threshold
import matplotlib.pyplot as plt
from nilearn.plotting import plot_stat_map
from nilearn.image import index_img











def build_epochs_evoked(raw):

    event_id = {"Auditory 300Hz": 6, "Auditory 600Hz": 7,
                "Auditory 1200Hz": 8, "Visual Checkerboard": 9}
    events = mne.find_events(raw, stim_channel='STI101')
    raw.info['projs'] = list()  # remove proj, don't proj while interpolating
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax,
                        baseline=(None, 0), reject=None,
                        verbose=False, detrend=0, preload=True)
    reject = get_rejection_threshold(epochs)
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax,
                        baseline=(None, 0), reject=reject,
                        verbose=False, detrend=0, preload=True)
    evoked = epochs["Auditory 300Hz"].average()
    print("Evoked for Auditory 300Hz : \n")
    evoked.plot()
    plt.show()

    evoked = epochs["Auditory 600Hz"].average()
    print("Evoked for Auditory 600Hz : \n")
    evoked.plot()
    plt.show()

    evoked = epochs["Auditory 1200Hz"].average()
    print("Evoked for Auditory 1200Hz : \n")
    evoked.plot()
    plt.show()

    evoked = epochs["Visual Checkerboard"].average()
    print("Evoked for Visual Checkerboard : \n")
    evoked.plot()
    plt.show()

    evoked = epochs.average()

    return (epochs, evoked)


def plot_sti_cols(raw):
    sti_cols = [i for i in range(len(raw.ch_names))
                if "STI" in raw.ch_names[i]]
    for i in sti_cols:
        print("Channel {} :".format(raw.ch_names[i]))
        plt.figure(figsize=(15, 2))
        plt.plot(list(range(len(raw._data[i]))), raw._data[i])
        x1, x2, y1, y2 = plt.axis()
        plt.show()


def compare_noise_cov(epochs, empty_raw):
    noise_cov_baseline = mne.compute_covariance(epochs, tmax=0)
    noise_cov = mne.compute_raw_covariance(empty_raw, tmin=0, tmax=None)
    noise_cov_reg = mne.compute_covariance(
        epochs, tmax=0., method='auto', rank=None)
    noise_cov.plot(empty_raw.info, proj=True)
    noise_cov_reg.plot(empty_raw.info, proj=True)
    noise_cov_baseline.plot(epochs.info, proj=True)
    print("Gotcha !")
    evoked = epochs.average()
    evoked.plot_white(noise_cov_reg, time_unit='s')


def build_dspm(raw):

    # Set dir
    data_path = "/storage/store/data/camcan-mne/freesurfer/"
    subjects_dir = data_path
    subject = "CC110033"
    bem_dir = op.join(subjects_dir, subject, 'bem')
    fname_aseg = op.join(subjects_dir, subject, 'mri', 'aseg.mgz')
    fname_model = op.join(bem_dir, 'CC110033-meg-bem.fif')

    # List substructures we are interested in. We select only the
    # sub structures we want to include in the source space
    labels_vol = ['Left-Amygdala',
                'Left-Thalamus-Proper',
                'Left-Cerebellum-Cortex',
                'Brain-Stem',
                'Right-Amygdala',
                'Right-Thalamus-Proper',
                'Right-Cerebellum-Cortex']

    # Get a surface-based source space, here with few source points for speed
    # in this demonstration, in general you should use oct6 spacing!
    src = mne.setup_source_space(subject, spacing='oct5',
                                add_dist=False, subjects_dir=subjects_dir)

    # Now we create a mixed src space by adding the volume regions specified in the
    # list labels_vol. First, read the aseg file and the source space bounds
    # using the inner skull surface (here using 10mm spacing to save time,
    # we recommend something smaller like 5.0 in actual analyses):

    vol_src = mne.setup_volume_source_space(
        subject, mri=fname_aseg, pos=5, bem=fname_model,
        volume_label=labels_vol, subjects_dir=subjects_dir,
        add_interpolator=True,  # just for speed, usually this should be True
        verbose=True)

    # Generate the mixed source space
    # src += vol_src

    # Visualize the source space.
    # src.plot(subjects_dir=subjects_dir)

    # n = sum(src[i]['nuse'] for i in range(len(src)))
    # print('the src space contains %d spaces and %d points' % (len(src), n))

    event_id = {"Auditory 300Hz": 6, "Auditory 600Hz": 7,
                "Auditory 1200Hz": 8, "Visual Checkerboard": 9}
    events = mne.find_events(raw, stim_channel='STI101')
    raw.info['projs'] = list()  # remove proj, don't proj while interpolating
    tmin, tmax = -0.2, 0.5
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax,
                        baseline=(None, 0), reject=None,
                        verbose=False, detrend=0, preload=True)
    reject = get_rejection_threshold(epochs)
    ica = mne.preprocessing.ICA(n_components=0.95, random_state=0).fit(raw, decim=1, reject=reject)
    eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None)
    eog_inds, eog_scores = ica.find_bads_eog(eog_epochs)
    ica.plot_scores(eog_scores, eog_inds)  # see scores the selection is based on
    # ica.plot_components(eog_inds)  # view topographic sensitivity of components
    ica.plot_overlay(eog_epochs.average())  # inspect artifact removal
    ica.apply(epochs)  # clean data, default in place
    evoked = [epochs[k].average() for k in event_id]
    for e in evoked:
        e.plot(ylim=dict(mag=[-400, 400]))

    plt.show()

    # estimate noise covarariance
    noise_cov = mne.compute_covariance(epochs, tmax=0, method='shrunk',
                                    rank=None)
    contrast = mne.combine_evoked(evoked, weights="equal") 
    # The transformation here was aligned using the dig-montage. It's included in
    # the spm_faces dataset and is named SPM_dig_montage.fif.
    trans_fname = '/storage/store/data/camcan-mne/trans/sub-CC110033-trans.fif'
    evoked[0].plot_white(noise_cov)
    # maps = mne.make_field_map(evoked[0], trans_fname, subject='spm',
    #                           subjects_dir='/storage/store/data/camcan-mne/trans/', n_jobs=1)

    # evoked[0].plot_field(maps, time=0.170)
    src = mne.setup_source_space("CC110033", spacing='oct5',
                                 add_dist=False, subjects_dir="/storage/store/data/camcan-mne/freesurfer/")
    bem = "/storage/store/data/camcan-mne/freesurfer/CC110033/bem/CC110033-meg-bem.fif"
    forward = mne.make_forward_solution(contrast.info, trans_fname, src, bem)
    snr = 3.0
    lambda2 = 1.0 / snr ** 2
    method = 'dSPM'

    inverse_operator = mne.minimum_norm.make_inverse_operator(contrast.info, forward, noise_cov,
                                            loose=0.2, depth=0.8)

    # Compute inverse solution on contrast

    stc = mne.minimum_norm.apply_inverse(contrast, inverse_operator, lambda2, method, pick_ori=None)
    # stc.save('spm_%s_dSPM_inverse' % contrast.comment)

    # Plot contrast in 3D with PySurfer if available
    subjects_dir = "/storage/store/data/camcan-mne/freesurfer/"
    # brain = stc.plot(hemi='both', subjects_dir=subjects_dir, initial_time=0.170,
    #                  views=['ven'], clim={'kind': 'value', 'lims': [3., 6., 9.]})
    stc.crop(0.0, 0.2)

    # Export result as a 4D nifti object
    img = stc.as_volume(inverse_operator["src"], mri_resolution=False)  # set True for full MRI resolution

    # Save it as a nifti file
    # nib.save(img, 'mne_%s_inverse.nii.gz' % method)

    t1_fname = "/storage/store/data/camcan-mne/freesurfer/CC110033/mri/T1.mgz"

    # Plotting with nilearn ######################################################
    plot_stat_map(index_img(img, 61), t1_fname, threshold=8., title='%s (t=%.1f s.)' % (method, stc.times[61]))


def ICA_preprocessing(raw):
    ica = mne.preprocessing.ICA(n_components=15, random_state=97)
    ica.fit(raw)
    raw.load_data()
    ica.plot_sources(raw)
    ica.plot_components()
    ica.exclude = []
    # find which ICs match the EOG pattern
    eog_indices, eog_scores = ica.find_bads_eog(raw)
    ica.exclude = eog_indices

    # barplot of ICA component "EOG match" scores
    ica.plot_scores(eog_scores)

    # plot diagnostics
    ica.plot_properties(raw, picks=eog_indices)

    # plot ICs applied to raw data, with EOG matches highlighted
    ica.plot_sources(raw)
    eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None)
    eog_evoked = eog_epochs.average()
    # plot ICs applied to the averaged EOG epochs, with EOG matches highlighted
    ica.plot_sources(eog_evoked)